In [3]:
import requests
import pandas as pd

url_count_json = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/comptages-routiers-permanents/exports/json?order_by=k&sort=desc&limit=500"


In [4]:
import pandas as pd
import json

def get_dpe_from_url(url):
    req = requests.get(url)
    
    # Décoder la réponse en bytes vers une chaîne
    decoded_content = req.content.decode("utf-8")
    
    # Charger le JSON à partir de la chaîne décodée
    wb = json.loads(decoded_content)
    # Vérifier si la clé 'results' existe
    if isinstance(wb, list):
            # Directly normalize the list of records into a DataFrame
            df = pd.DataFrame(wb)
    else:
            # If the response contains a 'results' key, normalize that
            df = pd.json_normalize(wb.get("results"))
    
    return df

In [5]:
dpe_count_json = get_dpe_from_url(url_count_json)

In [6]:
dpe_count_json.head(2)

,iu_ac,libelle,t_1h,q,k,etat_trafic,iu_nd_amont,libelle_nd_amont,iu_nd_aval,libelle_nd_aval,etat_barre,date_debut,date_fin,geo_point_2d,geo_shape
0,271,VGP_Cours_la_Reine,2023-11-06T08:00:00+00:00,853.0,0.0,Fluide,163,Souterrain_Cours_la_Reine,135,Cours_la_Reine-Concorde,Invalide,1996-10-03,2023-01-01,"{'lon': 2.3084567115959773, 'lat': 48.86467042...","{'type': 'Feature', 'geometry': {'coordinates'..."
1,4624,Souterrain_Alma_VGP,2023-11-06T04:00:00+00:00,NaN,0.0,Fluide,2447,Av_de_New_York-Debrousse,2467,Cours_Albert_Ier-VGP,Invalide,1996-10-16,2023-01-01,"{'lon': 2.301553929792855, 'lat': 48.864264855...","{'type': 'Feature', 'geometry': {'coordinates'..."


In [15]:
import geopandas as gpd

def get_geo_from_dpe_count(df):
    # Vérifiez si la colonne 'geo_point_2d' existe
    if "geo_point_2d" not in df.columns:
        raise ValueError("La colonne 'geo_point_2d' doit exister dans le DataFrame.")
    
    # Extraire longitude et latitude à partir des dictionnaires
    df["longitude"] = df["geo_point_2d"].apply(lambda x: x.get("lon"))
    df["latitude"] = df["geo_point_2d"].apply(lambda x: x.get("lat"))
    
    # Supprimer les lignes où les coordonnées sont manquantes
    df = df.dropna(subset=["longitude", "latitude"])
    
    # Créer un GeoDataFrame avec des points géographiques
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
        crs="EPSG:4326"  # Utilise WGS84 comme système de référence
    )
    
    return gdf

# Exemple d'appel
geo_count_j = get_geo_from_dpe_count(dpe_count_json)


In [17]:
geo_count_j.head(2)

,iu_ac,libelle,t_1h,q,k,etat_trafic,iu_nd_amont,libelle_nd_amont,iu_nd_aval,libelle_nd_aval,etat_barre,date_debut,date_fin,geo_point_2d,geo_shape,longitude,latitude,geometry
0,271,VGP_Cours_la_Reine,2023-11-06T08:00:00+00:00,853.0,0.0,Fluide,163,Souterrain_Cours_la_Reine,135,Cours_la_Reine-Concorde,Invalide,1996-10-03,2023-01-01,"{'lon': 2.3084567115959773, 'lat': 48.86467042...","{'type': 'Feature', 'geometry': {'coordinates'...",2.308457,48.864670,POINT (2.30846 48.86467)
1,4624,Souterrain_Alma_VGP,2023-11-06T04:00:00+00:00,NaN,0.0,Fluide,2447,Av_de_New_York-Debrousse,2467,Cours_Albert_Ier-VGP,Invalide,1996-10-16,2023-01-01,"{'lon': 2.301553929792855, 'lat': 48.864264855...","{'type': 'Feature', 'geometry': {'coordinates'...",2.301554,48.864265,POINT (2.30155 48.86426)
